# (WI4650) CodeLab 2 - QAOA
### Problem: QAOA for the maximum-cut problem

In this Codelab you will learn how to implement QAOA for the maximum-cut problem using Pennylane.

To get started, please do the following:
1. Install Pennylane and some additional Python packages: `pip install jupyterlab matplotlib pennylane seaborn pandas tqdm`
2. Start JubyterLab: `jupyter lab`

## A working QAOA example

Pennylane ships with all components required to implement a customized QAOA.

## Part 0 - Setup

First we setup out libraries, helper functions, and problem instance.

In [ ]:
from typing import List, Tuple

import networkx as nx
import pandas as pd
import pennylane as qml
import seaborn as sns
from matplotlib import pyplot as plt
from pennylane import numpy as np
from tqdm import tqdm

sns.set_theme()

In [ ]:
# Helper functions
def flatten(xss):
    return [x for xs in xss for x in xs]


# Specific to the (faster) lightning simulator
def bitstring_to_int(bit_string_sample):
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    return int(bit_string, base=2)

Let us create the weighted undirected graph $G=(V,E)$ with five vertices $V$ and edges $E$ from the lecture slides.

In [ ]:
edges: List[Tuple[int, int, float]] = [
    (0, 1, 1.0),
    (0, 2, 2.0),
    (1, 3, 3.0),
    (2, 3, 7.0),
    (2, 4, 4.0),
    (3, 4, 5.0),
]

graph = nx.Graph()
graph.add_nodes_from(list(range(5)))
graph.add_weighted_edges_from(edges)

In [ ]:
n_wires = len(set(flatten([[e[0], e[1]] for e in edges])))
dev = qml.device("lightning.qubit", wires=n_wires, shots=1)

In [ ]:
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True, font_weight="bold")
labels = nx.get_edge_attributes(graph, "weight")
nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
plt.show()

## Part 1 - Quantum Circuits

Next, your task is to set up the quantum circuits for the given problem.


### Assignment 1 - $H_M$
Create the time-evolution of the Mixer Hamiltonian $H_M = \sum_{j=1}^nX_i$. Implement a function `U_M` with parameter `beta` that creates the mixing Hamiltonian $U_M(\beta)$ for the max-cut problem. You should use the Pennylane API, documented [here](https://docs.pennylane.ai/en/stable/code/qml.html). For this part, you might find operations like the [RX](https://docs.pennylane.ai/en/stable/code/api/pennylane.RX.html) useful. Note that you have access to the global variable `edges`.

In [ ]:
def U_M(beta: float):
    # Your code here!
    pass

### Assignment 2 - $H_C$

Create the time-evolution of the Cost Hamiltonian $H_C = \frac{1}{2}\sum_{(j,k)\in E} W_{jk}(\mathbb{I} - Z_jZ_k)$. Implement a function `U_C` with parameter `gamma` that creates the Hamiltonian $U_C(\gamma)$ for the max-cut problem. You should use the Pennylane API, documented [here](https://docs.pennylane.ai/en/stable/code/qml.html). For this part, you might find operations like the [RZ](https://docs.pennylane.ai/en/stable/code/api/pennylane.RZ.html) and [CNOT](https://docs.pennylane.ai/en/stable/code/api/pennylane.CNOT.html) useful. Note that you have access to the global variable `edges`.

In [ ]:
def U_C(gamma: float):
    # Your code here!
    pass

### Assignment 3 - $H_C$ encoding

Consider the following sum of Pauli strings:

$IIIIIIZZ + IIIIIZIZ + IIIZIIIZ + IIIIIZZI + IIIIZZII + IIIZZIII + IIZZIIII + IZZIIIII + ZIZIIIII$

Assuming this is the encoding of a MaxCut Cost Hamiltonian $H_C = \frac{1}{2}\sum_{(j,k)\in E} W_{jk}(\mathbb{I} - Z_jZ_k)$, derive the structure of the underlying graph. You can either provide an adjacency list or draw the graph as in the cells above.

Your answer here...

In [ ]:
# Or here...

### Assignment 4 - QAOA

Implement the $\prod_{k=1}^p e^{-iU_C(\boldsymbol{\beta}_k)}e^{-iU_M(\boldsymbol{\gamma}_k)}$ circuit that evolves $\ket{\psi(0)}$ into $\ket{\psi(\boldsymbol{\gamma}, \boldsymbol{\beta}})$. Implement a function `circuit` with parameters `gammas` and `betas` that uses $U_C(\gamma)$ and $U_M(\beta)$ (in this order) for the max-cut problem. You should use the Pennylane API, documented [here](https://docs.pennylane.ai/en/stable/code/qml.html). Remember the initial state is $\ket{0}^{\otimes n}$.

In [ ]:
@qml.qnode(dev)
def circuit(gammas, betas, edge=None, n_layers=1):
    # Your code here!

    if edge is None:
        # measurement phase
        return qml.sample()

    # during the optimization phase we are evaluating a term
    # in the objective using expval
    H = qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])
    return qml.expval(H)

## Part 3 - Experimentation

We provide an optimization loop for you to test your implementation on. Feel free to experiment with different parameter settings and circuit implementations!

In [ ]:
def qaoa_maxcut(
    n_layers: int = 1, opt=qml.AdagradOptimizer(stepsize=0.5), steps: int = 100
):
    print("\np={:d}".format(n_layers))

    # minimize the negative of the objective function
    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = 0
        for edge in edges:
            # objective for the MaxCut problem
            neg_obj -= 0.5 * (1 - circuit(gammas, betas, edge=edge, n_layers=n_layers))
        return neg_obj

    # optimize parameters in objective
    result_dict = {"step": [], "objective": [], "layers": []}

    # Sample some parameters
    params = 0.01 * np.random.rand(2, n_layers, requires_grad=True)

    for i in tqdm(range(steps)):
        params = opt.step(objective, params)
        obj_value = -objective(params)
        result_dict["step"].append(i + 1)
        result_dict["objective"].append(obj_value)
        result_dict["layers"].append(n_layers)

    # sample measured bitstrings 100 times
    bit_strings = []
    n_samples = 100
    for i in range(0, n_samples):
        bit_strings.append(
            bitstring_to_int(
                circuit(params[0], params[1], edge=None, n_layers=n_layers)
            )
        )

    # print optimal parameters and most frequently sampled bitstring
    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print("Optimized (gamma, beta) vectors:\n{}".format(params[:, :n_layers]))
    print(
        "Most frequently sampled bit string is:",
        format(most_freq_bit_string, f"0{n_wires}b"),
    )

    return -objective(params=params), bit_strings, result_dict

In [ ]:
obj1, bitstrings1, res1 = qaoa_maxcut(n_layers=1, steps=50)
obj2, bitstrings2, res2 = qaoa_maxcut(n_layers=2, steps=50)

## Part 4 - Analysis

Now that we hve some empirical data, we can analyze the differences between the the two QAOA versions. We begin by preparing the data.

In [ ]:
df = pd.concat([pd.DataFrame.from_dict(res1), pd.DataFrame.from_dict(res2)])
df

Next, we can visualize the results.

In [ ]:
sns.lineplot(
    data=df, x="step", y="objective", hue="layers", style="layers", markers=True
)
plt.xticks(sorted([x for x in set(df["step"]) if x % 10 == 0]))
plt.show()

In [ ]:
xticks = range(0, 32)
xtick_labels = list(map(lambda x: format(x, f"0{n_wires}b"), xticks))
bins = np.arange(0, 33) - 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
plt.subplot(1, 2, 1)
plt.title("n_layers=1")
plt.xlabel("bitstrings")
plt.ylabel("freq.")
plt.xticks(xticks, xtick_labels, rotation="vertical")
plt.hist(bitstrings1, bins=bins)
plt.subplot(1, 2, 2)
plt.title("n_layers=2")
plt.xlabel("bitstrings")
plt.ylabel("freq.")
plt.xticks(xticks, xtick_labels, rotation="vertical")
plt.hist(bitstrings2, bins=bins)
# plt.tight_layout()
plt.show()

## Assignment 5 - Analysis

Discuss the results obtained in the experiment above. Which bit strings are sampled most frequently and how do they encode? How does the additional layer affect the results? Feel free to consider additional variables and plot your own data.

Your answer here...

## Assignment 6 - Mixing Hamiltonian

The role of the Mixing Hamiltonian does not clearly follow from the derivation of the QUBO formulation. Explain why $H_M$ is necessary (not necessarily the specific version you are asked to implement in the assignment) for QAOA to work for the MaxCut problem.

You could approach this in two ways: either analytically explain the mathematics of the systems with and without $H_M$ or empirically show the effects of removing $U_M$ from the `circuit` function. You are encouraged to experiment with both!

Your answer here...

In [ ]:
# Or here...